In [39]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow_datasets as tfds
import tensorflow as tf
tf.__version__

'2.12.0'

In [68]:
mail= pd.read_csv('https://full-stack-bigdata-datasets.s3.eu-west-3.amazonaws.com/Deep+Learning/project/spam.csv', encoding="ISO-8859-1")

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [41]:
print("Number of rows : {}".format(mail.shape[0]))
print()

print("Display of dataset: ")
display(mail.head())
print()

print("Basics statistics: ")
data_desc = mail.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*mail.isnull().sum()/mail.shape[0])

Number of rows : 5572

Display of dataset: 


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN



Basics statistics: 


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2



Percentage of missing values: 


v1             0.000000
v2             0.000000
Unnamed: 2    99.102656
Unnamed: 3    99.784637
Unnamed: 4    99.892319
dtype: float64

In [42]:
useless_cols = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']

print("Dropping useless columns...")
mail = mail.drop(useless_cols, axis=1)

print("...Done.")
mail.head()


Dropping useless columns...
...Done.


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [43]:
import plotly.express as px

fig = px.histogram(mail, x= 'v1')
fig.show()

In [44]:
mail['mail_clean']= mail['v2'].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" "))
mail['mail_clean']= mail['mail_clean'].apply(lambda x: x.replace(r"[!\"#$%&()*+,-./:;<=>?@[\\\]^_`{|}~]+", " ").lower().strip())
mail['mail_clean']= mail['mail_clean'].str.strip()
mail.head()

,v1,v2,mail_clean
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...


In [45]:
!python -m spacy download en_core_web_sm -q

2023-08-07 12:50:07.721755: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [46]:
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.lang.en.stop_words import STOP_WORDS

In [47]:
mail['mail_clean'] = mail['mail_clean'].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) and (token.text not in STOP_WORDS)]))
mail.head()

,v1,v2,mail_clean
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun early hor u c
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think usf live


In [48]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="out_of_vocab")

tokenizer.fit_on_texts(mail['mail_clean'])

mail["mail_clean_encoded"] = tokenizer.texts_to_sequences(mail['mail_clean'])
mail.head()

,v1,v2,mail_clean,mail_clean_encoded
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...,"[1, 230, 447, 463, 942, 36, 52, 204, 943, 79, ..."
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni,"[10, 195, 464, 289, 2, 1]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,"[13, 299, 4, 533, 664, 34, 1, 851, 423, 1, 1, ..."
3,ham,U dun say so early hor... U c already then say...,u dun early hor u c,"[2, 125, 150, 1, 2, 85]"
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think usf live,"[709, 23, 665, 129]"


In [49]:
mail['v1_encoded'] = mail['v1'].apply(lambda x: 0 if x == 'ham' else 1)
mail.head()

,v1,v2,mail_clean,mail_clean_encoded,v1_encoded
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...,"[1, 230, 447, 463, 942, 36, 52, 204, 943, 79, ...",0
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni,"[10, 195, 464, 289, 2, 1]",0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,"[13, 299, 4, 533, 664, 34, 1, 851, 423, 1, 1, ...",1
3,ham,U dun say so early hor... U c already then say...,u dun early hor u c,"[2, 125, 150, 1, 2, 85]",0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think usf live,"[709, 23, 665, 129]",0


In [50]:
mail.shape

(5572, 5)

In [51]:
mail_pad = tf.keras.preprocessing.sequence.pad_sequences(mail.mail_clean_encoded, padding="post")

In [52]:
from sklearn.model_selection import train_test_split

X_train, X_valtest, y_train, y_valtest = train_test_split(mail_pad,mail.v1_encoded, test_size=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, test_size=0.5)

In [53]:
train_batch = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(64)
test_batch = tf.data.Dataset.from_tensor_slices((X_test, y_test)).shuffle(len(X_test)).batch(64)
val_batch = tf.data.Dataset.from_tensor_slices((X_val, y_val)).shuffle(len(X_val)).batch(64)

In [54]:
for mail, v1 in train_batch.take(1):
  print(mail, v1)

tf.Tensor(
[[  1   1 272 ...   0   0   0]
 [ 77 157   0 ...   0   0   0]
 [  1  83 328 ...   0   0   0]
 ...
 [259 208   0 ...   0   0   0]
 [  1 103 264 ...   0   0   0]
 [  2  88   1 ...   0   0   0]], shape=(64, 72), dtype=int32) tf.Tensor(
[0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0
 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0], shape=(64,), dtype=int64)


In [55]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.losses import BinaryCrossentropy
from keras.layers import Dropout

In [56]:
embedding_dim=8
vocab_size = 1000
model = Sequential([
  Embedding(vocab_size +1, embedding_dim, input_shape= [mail.shape[1]], name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1, activation="sigmoid")
])

In [57]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 72, 8)             8008      
                                                                 
 global_average_pooling1d_1   (None, 8)                0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 16)                144       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 8,169
Trainable params: 8,169
Non-trainable params: 0
_________________________________________________________________


In [58]:
model.compile(optimizer= 'adam',
              loss=BinaryCrossentropy(),
              metrics=['accuracy'])

In [59]:
history = model.fit(train_batch, epochs=50, validation_data=test_batch)

Epoch 1/50
53/53 [==============================] - 1s 6ms/step - loss: 0.6373 - accuracy: 0.8465 - val_loss: 0.5620 - val_accuracy: 0.8771
Epoch 2/50
53/53 [==============================] - 0s 2ms/step - loss: 0.4952 - accuracy: 0.8609 - val_loss: 0.4145 - val_accuracy: 0.8771
Epoch 3/50
53/53 [==============================] - 0s 2ms/step - loss: 0.3964 - accuracy: 0.8609 - val_loss: 0.3524 - val_accuracy: 0.8771
Epoch 4/50
53/53 [==============================] - 0s 2ms/step - loss: 0.3680 - accuracy: 0.8609 - val_loss: 0.3371 - val_accuracy: 0.8771
Epoch 5/50
53/53 [==============================] - 0s 4ms/step - loss: 0.3557 - accuracy: 0.8609 - val_loss: 0.3264 - val_accuracy: 0.8771
Epoch 6/50
53/53 [==============================] - 0s 3ms/step - loss: 0.3429 - accuracy: 0.8609 - val_loss: 0.3148 - val_accuracy: 0.8771
Epoch 7/50
53/53 [==============================] - 0s 2ms/step - loss: 0.3283 - accuracy: 0.8609 - val_loss: 0.3009 - val_accuracy: 0.8771
Epoch 8/50
53/53 [==

In [60]:
model.save('/content/spam_detector.h5')

In [61]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [62]:
fig = go.Figure(data=[go.Scatter(
                          y=history.history["loss"],
                          name="Training loss",
                          mode="lines"
                          ),
                      go.Scatter(
                          y=history.history["val_loss"],
                          name="Validation loss",
                          mode="lines"
                          )
])
fig.update_layout(
    title='Training and val loss across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'
)
fig.show()


In [63]:
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["accuracy"],
                          name="Training accuracy",
                          mode="lines"
                          ),
                      go.Scatter(
                          y=history.history["val_accuracy"],
                          name="Validation accruracy",
                          mode="lines")])
fig.update_layout(
    title='Training and val accuracy across epochs',
    xaxis_title='epochs',
    yaxis_title='Accuracy'
)
fig.show()

In [64]:
from keras.models import load_model

model = load_model('/content/spam_detector.h5')

In [65]:
history = model.fit(train_batch, epochs=50, validation_data=val_batch)
loss, accuracy = model.evaluate(test_batch)

Epoch 1/50
53/53 [==============================] - 2s 14ms/step - loss: 0.0217 - accuracy: 0.9934 - val_loss: 0.0388 - val_accuracy: 0.9874
Epoch 2/50
53/53 [==============================] - 0s 5ms/step - loss: 0.0216 - accuracy: 0.9946 - val_loss: 0.0389 - val_accuracy: 0.9856
Epoch 3/50
53/53 [==============================] - 0s 6ms/step - loss: 0.0213 - accuracy: 0.9946 - val_loss: 0.0388 - val_accuracy: 0.9874
Epoch 4/50
53/53 [==============================] - 0s 5ms/step - loss: 0.0206 - accuracy: 0.9943 - val_loss: 0.0388 - val_accuracy: 0.9874
Epoch 5/50
53/53 [==============================] - 0s 6ms/step - loss: 0.0208 - accuracy: 0.9946 - val_loss: 0.0402 - val_accuracy: 0.9856
Epoch 6/50
53/53 [==============================] - 0s 5ms/step - loss: 0.0196 - accuracy: 0.9949 - val_loss: 0.0393 - val_accuracy: 0.9856
Epoch 7/50
53/53 [==============================] - 0s 6ms/step - loss: 0.0194 - accuracy: 0.9952 - val_loss: 0.0393 - val_accuracy: 0.9892
Epoch 8/50
53/53 [=

In [66]:
fig = go.Figure(data=[go.Scatter(
                          y=history.history["loss"],
                          name="Training loss",
                          mode="lines"
                          ),
                      go.Scatter(
                          y=history.history["val_loss"],
                          name="Validation loss",
                          mode="lines"
                          )
])
fig.update_layout(
    title='Training and val loss across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'
)
fig.update_yaxes(range=[0, 0.6])
fig.show()

In [67]:
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["accuracy"],
                          name="Training accuracy",
                          mode="lines"
                          ),
                      go.Scatter(
                          y=history.history["val_accuracy"],
                          name="Validation accruracy",
                          mode="lines")])
fig.update_layout(
    title='Training and val accuracy across epochs',
    xaxis_title='epochs',
    yaxis_title='Accuracy'
)

fig.update_yaxes(range=[0.85, 1])
fig.show()